In [1]:
from sklearn.metrics import pairwise_distances
import math
import pandas as pd
import numpy as np
import warnings
from collections import Counter
warnings.filterwarnings("ignore")

data=pd.read_pickle(r"16k_apperal_data")
print(data)

              asin                        brand                 color  \
4       B004GSI2OS                  FeatherLite     Onyx Black/ Stone   
6       B012YX2ZPI  HX-Kingdom Fashion T-shirts                 White   
15      B003BSRPB0                  FeatherLite                 White   
27      B014ICEJ1Q                        FNC7C                Purple   
46      B01NACPBG2                 Fifth Degree                 Black   
60      B014ICB9A0                        FNC7C                 Black   
78      B011YPK0MW               Feel The Piece                  Navy   
95      B0079QXJ3S                FineBrandShop               Fuchsia   
107     B00R7DO9ZA                       Finejo                 white   
121     B011JQWCCM                      Ferrari                   Red   
133     B01NAZ3L3C                 Fifth Degree                 Black   
138     B01I5GRO18               FIFTEEN TWENTY              Eggshell   
139     B002ZZT446          Florida Hat Company    

In [2]:
import time
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [3]:
stop_words=set(stopwords.words('english'))

In [4]:
print(stop_words)

{'at', 'if', 'being', 'over', 'ma', 'whom', 'off', 'while', 'am', 'why', 'between', 'are', 'when', 'herself', 'doing', 'most', 'aren', 'those', 't', 'a', 'hasn', 'do', 'theirs', 'just', 'who', 'o', 'each', 'were', 'too', 'before', 'ain', 'ours', 'and', 'some', 'won', 'weren', 'd', 'i', 'out', 'nor', 'had', 'which', 'to', 'in', 'that', 'up', 'under', 'yourselves', 'her', 'wouldn', 'through', 'very', 'yours', 'hadn', 'did', 'itself', 're', 'himself', 'after', 'for', 'so', 'any', 'here', 'all', 'hers', 'your', 'he', 'its', 'this', 'it', 'an', 'again', 'is', 'than', 'haven', 'can', 'into', 'them', 'isn', 'was', 'below', 'doesn', 'myself', 'have', 'no', 'shan', 'has', 'as', 'from', 've', 'm', 'wasn', 'our', 'their', 'themselves', 'they', 'then', 's', 'or', 'but', 'having', 'more', 'only', 'now', 'on', 'these', 'both', 'she', 'him', 'me', 'above', 'such', 'by', 'where', 'will', 'further', 'shouldn', 'own', 'until', 'don', 'y', 'you', 'there', 'what', 'with', 'my', 'few', 'once', 'couldn', 'd

In [5]:
def nlp_preprocessing(total_text,index,column):
    if type(total_text) is not int:
        string=" "
        for word in total_text.split():
            word=(''.join(e for e in word if e.isalnum()))
            word=word.lower()
            if not word in stop_words:
                string+= word + " "
        data[column][index]= string 
starttime=time.clock()
for index, row in data.iterrows():
    nlp_preprocessing(row['title'],index,'title')
et=time.clock()  

In [6]:
print(data)

              asin                        brand                 color  \
4       B004GSI2OS                  FeatherLite     Onyx Black/ Stone   
6       B012YX2ZPI  HX-Kingdom Fashion T-shirts                 White   
15      B003BSRPB0                  FeatherLite                 White   
27      B014ICEJ1Q                        FNC7C                Purple   
46      B01NACPBG2                 Fifth Degree                 Black   
60      B014ICB9A0                        FNC7C                 Black   
78      B011YPK0MW               Feel The Piece                  Navy   
95      B0079QXJ3S                FineBrandShop               Fuchsia   
107     B00R7DO9ZA                       Finejo                 white   
121     B011JQWCCM                      Ferrari                   Red   
133     B01NAZ3L3C                 Fifth Degree                 Black   
138     B01I5GRO18               FIFTEEN TWENTY              Eggshell   
139     B002ZZT446          Florida Hat Company    

In [7]:
print(et-starttime,'second')

4.102873551194198 second


In [8]:
from sklearn.feature_extraction.text import CountVectorizer
title_vectorizer=CountVectorizer()




In [9]:
from PIL import Image
import requests
from io import BytesIO
import matplotlib.pyplot as plt
import re
import os
import seaborn as sns
from matplotlib import gridspec
#import plotly
#import plotly.figure_factory as ff
def display_img(url,ax,fig):
    response=requests.get(url)
    img=Image.open(BytesIO(response.content))
    plt.imshow(img)

def plot_heatmap(keys,values,labels,url,text):
    gs=gridspec.GridSpec(2,2,width_ratios=[4,1],hight_ratios=[41])
    fig=plt.figure(figsize=(25,3))
    ax=plt.subplot(gs[0])
    ax=sns.heatmap(np.array([values]),annot=np.array([labels]))
    ax.set_xticklabels(keys)
    ax.set_title(text)
    ax=plt.subplot(gs[1])
    ax.grid(False)
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    display_img(url,ax,fig)
    plt.show()
    
def plot_heatmap_image(doc_id,vec1,vec2,url,text,model):
    intersections=set(vec1.keys()) & set(vec2.keys())
    for i in vec2:
        if i not in intersections:
            vec2[i]=0
    keys=list(vec2.keys())
    values=[vec2[x] for x in vec2.keys()]
    if model=='bag_of_word':
        lables=values
    elif model=='tfidf':
        labels=[]
        for x in vec2.keys():
            if x in tfidf_title_vectorizer.vocabulary_:
                labels.append(tfidf_title_features[doc_id,tfidf_title_vectorizer])
            else:
                labels.append(0)
    elif model =='idf':
        labels=[]
        for x in vec2.keys():
            if x in tfidf_title_vectorizer.vocabulary_:
                labels.append(tfidf_title_features[doc_id,tfidf_title_vectorizer])
            else:
                labels.append(0)
    plot_heatmap(keys,values,labels,url,text)  
    
def text_to_vector(text):
    word=re.compile(r'\w+')
    words=word.findall(text)
    return Counter(words)

def get_result(doc_id,content_a,content_b,url,model):
    text1=content_a
    text2=content_b
    vector1=text_to_vector(text1)
    vector2=text_to_vector(text2)
    plot_heatmap_image(doc_id,vector1,vector2,url,text2,model)
    
    

In [10]:
print(data['title'])

4          featherlite ladies long sleeve stain resistan...
6          womens unique 100 cotton  special olympics wo...
15         featherlite ladies moisture free mesh sport s...
27         supernatural chibis sam dean castiel neck tsh...
46         fifth degree womens gold foil graphic tees ju...
60         supernatural chibis sam dean castiel neck shi...
78               feel piece sami dip dye top one size navy 
95         ladies fuchsia pink seamless stone set tube top 
107        finejo ladies sleeveless shirt lace shirts hi...
121        puma womens ferrari big shield tee rosso cors...
133        vein womens angels diamonds 34 sequin sleeve ...
138        fifteentwenty womens shirred hem ruffle blous...
139                        short purple gloves  satin feel 
147        womens west ham united fc logo short sleeve t...
169        featherlite ladies pique sport shirt forest g...
179        feel piece womens super soft turtleneck curve...
182          fjallraven  womens alphabot

In [11]:
title_feature=title_vectorizer.fit_transform(data['title'])

In [12]:
title_feature.get_shape()

(16042, 12609)

In [13]:
# using bag of words method
def bag_of_word(doc_id,no_of_result):
    pairwise_dis=pairwise_distances(title_feature,title_feature[doc_id])
    indecs=np.argsort(pairwise_dis.flatten())[0:no_of_result]
    pdists=np.sort(pairwise_dis.flatten())[0:no_of_result]
    df_indices=data.index[indecs]
    for i in range(0,len(indecs)):
       # get_result(indecs[i],data['title'].loc[df_indices[0]],data['title'])
        print('ASIN:',data['asin'].loc[df_indices[i]])
        print('Brand:',data['brand'].loc[df_indices[i]])
        print('Title:',data['title'].loc[df_indices[i]])
        print('euclidean similarity with the quiry image',pdists[i])
        print(' '*60)

In [14]:
bag_of_word(12566,10)

ASIN: B00JXQB5FQ
Brand: Si Row
Title:  burnt umber tiger tshirt zebra stripes xl  xxl 
euclidean similarity with the quiry image 0.0
                                                            
ASIN: B00JXQASS6
Brand: Si Row
Title:  pink tiger tshirt zebra stripes xl  xxl 
euclidean similarity with the quiry image 1.73205080757
                                                            
ASIN: B00JXQCWTO
Brand: Si Row
Title:  brown  white tiger tshirt tiger stripes xl  xxl 
euclidean similarity with the quiry image 2.44948974278
                                                            
ASIN: B00JXQCUIC
Brand: Si Row
Title:  yellow tiger tshirt tiger stripes  l 
euclidean similarity with the quiry image 2.64575131106
                                                            
ASIN: B07568NZX4
Brand: Rustic Grace
Title:  believed could tshirt 
euclidean similarity with the quiry image 3.0
                                                            
ASIN: B01NB0NKRO
Brand: Ideology
Ti

# using the tfidf method

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import hstack
tfidf_title_vectorizer=TfidfVectorizer(min_df=0)
tfidf_title_feature=tfidf_title_vectorizer.fit_transform(data['title'])

def tf_idf(doc_id,no_of_result):
    pairwise_dis=pairwise_distances(tfidf_title_feature,tfidf_title_feature[doc_id])
    indecs=np.argsort(pairwise_dis.flatten())[0:no_of_result]
    pdists=np.sort(pairwise_dis.flatten())[0:no_of_result]
    df_indices=data.index[indecs]
    for i in range(0,len(indecs)):
       # get_result(indecs[i],data['title'].loc[df_indices[0]],data['title'])
        print('ASIN:',data['asin'].loc[df_indices[i]])
        print('Brand:',data['brand'].loc[df_indices[i]])
        print('Title:',data['title'].loc[df_indices[i]])
        print('euclidean similarity with the quiry image',pdists[i])
        print('='*125)
tf_idf(12566,20)        

ASIN: B00JXQB5FQ
Brand: Si Row
Title:  burnt umber tiger tshirt zebra stripes xl  xxl 
euclidean similarity with the quiry image 0.0
ASIN: B00JXQASS6
Brand: Si Row
Title:  pink tiger tshirt zebra stripes xl  xxl 
euclidean similarity with the quiry image 0.753633191245
ASIN: B00JXQCWTO
Brand: Si Row
Title:  brown  white tiger tshirt tiger stripes xl  xxl 
euclidean similarity with the quiry image 0.935764394377
ASIN: B00JXQAFZ2
Brand: Si Row
Title:  grey  white tiger tank top tiger stripes xl  xxl 
euclidean similarity with the quiry image 0.95861535242
ASIN: B00JXQCUIC
Brand: Si Row
Title:  yellow tiger tshirt tiger stripes  l 
euclidean similarity with the quiry image 1.00007496145
ASIN: B00JXQAO94
Brand: Si Row
Title:  black  white tiger tank top tiger stripes  l 
euclidean similarity with the quiry image 1.02321555246
ASIN: B00JXQAUWA
Brand: Si Row
Title:  yellow tiger tank top tiger stripes  l 
euclidean similarity with the quiry image 1.0319918463
ASIN: B06XSCVFT5
Brand: Studio M

# uding the word2vec method 

In [16]:
bottelneck_features_train=np.load('16k_data_cnn_features.npy')   
asian=np.load('16k_data_cnn_feature_asins.npy')
asian=list(asian)
data=pd.read_pickle('16k_apperal_data_preprocessed')
print(data)
df_asin=list(data['asin'])
from IPython.display import display, Image, SGV, Math, YouTubeVideo

#get similar product using CNN features using VGG-16
def get_similar_product_cnn(doc_id,num_results):
    doc_id=asin.index(df_asin[doc_id])
    pairwise_dist=pairwise_distance(bottelneck_features_train,bottelneck_features_train[doc_id])

ValueError: cannot reshape array of size 396885996 into shape (16042,25088)